In [1]:
# import warnings
# warnings.filterwarnings('ignore')
import numpy as np
# np.set_printoptions(threshold=np.nan)
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
# import sklearn
# from sklearn import svm
from sklearn.svm import SVC
# from sklearn.grid_search import GridSearchCV
from sklearn.model_selection import GridSearchCV
import time
from sklearn import preprocessing
import csv
import time

In [10]:
%cd "C:\Users\rossiadmin\Dropbox (Stuber Lab)\Mark\LHA projection paper\Data\Fig4 ephys"
data=pd.read_excel('Summary_for_svm_truncated.xlsx')

C:\Users\rossiadmin\Dropbox (Stuber Lab)\Mark\LHA projection paper\Data\Fig4 ephys


In [12]:
## SVM function ###
def ephys_svm(filename,predict,params,iterations,test_data,df):
    parameters = [{'C': [.001,.01,.1,1,10,100,1000], 'kernel': ['linear']},
                  {'C': [.001,.01,.1,1,10,100,1000], 'gamma': [.001,.01,.1,1,10,100,1000], 'kernel': ['rbf']}]  
#     df=summarydata[summarydata.label != label_tag]

    ##normalize columns (except for 'predict' column)##
    df_norm=df[params]
    df_predict=df[predict]
    df_norm=pd.DataFrame(preprocessing.scale(df_norm))
    df_predict.reset_index(drop=True, inplace=True)
    df_norm.reset_index(drop=True, inplace=True)
    df=pd.concat([df_norm,df_predict],axis=1)
    df=df.dropna()
    headers=params+predict
    df.columns=headers

#     combined=params+predict
#     df=df[combined].dropna()
    
    pred_score=np.nan*np.zeros((iterations))
    rand_score=np.nan*np.zeros((iterations))
    for i in range(iterations):
        print '\nRun #: ', i+1
        ## if splits result in insufficient data to classify, skip run and move on ##
        try:
            ##randomly subset df into test and train sets
            msk = np.random.rand(len(df))>test_data
            train = df[msk]
            ##use gridsearch to select optimal parameters for each run
            clf = GridSearchCV(SVC(), parameters,cv=10,n_jobs=-1,iid=False)
            train_labels=train[predict].values
            c,r=train_labels.shape
            train_labels=train_labels.reshape(c,)
            clf.fit(train[params], train_labels)
            print 'prediction accuracy = ', clf.best_score_
            pred_score[i]=(clf.best_score_)*100
            ##random array of 1s and 0s for testing classifier
            rand_group=pd.DataFrame(np.random.randint(2,size=train.shape[0]))
            clf.fit(train[params], rand_group[0])
            print 'shuffled accuracy = ', clf.best_score_
            rand_score[i]=(clf.best_score_)*100
        except Exception:
            pass
    plt.figure(figsize=(5,5))
    ax=plt.subplot(111)
    n = np.arange(1,len(pred_score)+1) / np.float(len(pred_score))
    Xs = np.sort(pred_score)
    plt.step(Xs,n,color='g',label='Model prediction',alpha=0.7,linewidth=2)
    n2= np.arange(1,len(rand_score)+1) / np.float(len(rand_score))
    Xs2= np.sort(rand_score)
    plt.step(Xs2,n,color='r',label='Random',alpha=0.7,linewidth=2)
#     ax.set_axis_bgcolor('white')
    plt.xlabel('Decoding accuracy %',fontsize=14)
    plt.ylabel('Proportion of runs',fontsize=14)
    plt.legend(fontsize=14,loc=2)
    timestr = time.strftime("%Y%m%d-%H%M%S")
    
    plt.savefig(filename+'_'+timestr+'_'+str(iterations)+str('iter_')+str(test_data)+'test_data_prediction_accuracy_CDF.pdf')
    np.savetxt(filename+'_'+timestr+'_'+str(iterations)+str('iter_')+str(test_data)+'_prediction_accuracy.csv',pred_score,delimiter=',')
    np.savetxt(filename+'_'+timestr+'_'+str(iterations)+str('iter_')+str(test_data)+'_RANDOM_prediction_accuracy.csv',rand_score,delimiter=',')
    plt.show()
    
    #save parameters to csv
    master_parameters=['Classify on '+ str(params), '\nPredict '+str(predict), '\nCells used = '+str(condition[0]),
               '\nIterations = '+str(iterations), '\nProportion test data = '+str(test_data), 
                '\nSearch Parameters = '+str(parameters)]
    with open(filename+'_'+timestr+'_parameters','wb') as myfile:
        out=csv.writer(myfile,delimiter=' ')
        out.writerow(master_parameters)
    print "mean decoding accuracy = ", np.nanmean(pred_score)
    print "mean RANDOM decoding accuracy = ",np.nanmean(rand_score)
#     return pred_score,rand_score

In [24]:
## Fig 4K and S6I(fed): SVM decoding - use ad lib excitability params to decode projection target

## set SVM params, choose params to classify on, and subset cells ##

#### EDIT THIS ####
filename='Ad_lib_predict_projection'
predict=['projection'] ## variable to classify (e.g. projection, group, condition)
condition=['ad lib'] ## only used for reference in parameter file
params=['spikes/min whole cell','rheo (pA)','AP Thresh','AP duration (s)','APH dur','sum spikes'] 

summarydata=pd.DataFrame(data)
summarydata=summarydata.replace(['LHb','VTA'],[1,0])
summarydata=summarydata.replace(['ad lib','fasted'],[0,1])
# label_tag=0  ## Choose 1 for ad lib and 0 for fasted; or 1 for VTA and 0 for LHb ##

df=summarydata[summarydata.Condition == 0]  #change param as input data

iterations=1000
test_data=.2  #defines split for test/train sets
#####

### Run SVM for eYFP+ cells to predict group based on ephys params ###
start = time.time()
ephys_svm(filename,predict,params,iterations,test_data,df)
end = time.time()
print(end - start)

In [ ]:
%cd "C:\Users\rossiadmin\Dropbox (Stuber Lab)\Mark\LHA projection paper\Data\FigS6 ephys supp"

In [26]:
## Fig S6I (Fast) ##

#### EDIT THIS ####
filename='Fasted_predict_projection'
predict=['projection'] ## variable to classify (e.g. projection, group, condition)
condition=['fasted'] ## only used for reference in parameter file
params=['spikes/min whole cell','rheo (pA)','AP Thresh','AP duration (s)','APH dur','sum spikes'] 

summarydata=pd.DataFrame(data)
summarydata=summarydata.replace(['LHb','VTA'],[1,0])
summarydata=summarydata.replace(['ad lib','fasted'],[0,1])
# label_tag=1  ## Choose 1 for ad lib and 0 for fasted; or 1 for VTA and 0 for LHb ##
df=summarydata[summarydata.Condition == 1]  #change param as input data
iterations=1000
test_data=.2  #defines split for test/train sets
#####

### Run SVM for eYFP+ cells to predict group based on rheo and max latency ###
start = time.time()
ephys_svm(filename,predict,params,iterations,test_data,df)
end = time.time()
print(end - start)

In [30]:
## Fig S6I (All cells) ##

#### EDIT THIS ####
filename='All_Cells_predict_satiety_ALL_PARAMS'
predict=['projection'] ## variable to classify (e.g. projection, group, condition)
condition=['All Cells'] ## only used for reference in parameter file
params=['spikes/min whole cell','rheo (pA)','AP Thresh','AP duration (s)','APH dur','sum spikes'] 

summarydata=pd.DataFrame(data)
summarydata=summarydata.replace(['LHb','VTA'],[1,0])
summarydata=summarydata.replace(['ad lib','fasted'],[0,1])
# label_tag=1  ## Choose 1 for ad lib and 0 for fasted; or 1 for VTA and 0 for LHb ##

# df=summarydata[summarydata.Condition == 1]  #change param as input data
df=summarydata

iterations=1000
test_data=.2  #defines split for test/train sets
#####

### Run SVM for eYFP+ cells to predict group based on rheo and max latency ###
start = time.time()
ephys_svm(filename,predict,params,iterations,test_data,df)
end = time.time()
print(end - start)